<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

# Data Formatting (strings and numeric)

Let me clean some data sets:

In [28]:
# links to websites
carbonLink="https://www.cia.gov/the-world-factbook/field/carbon-dioxide-emissions/country-comparison" 
forestLink="https://www.cia.gov/the-world-factbook/field/revenue-from-forest-resources/country-comparison" 

# scrapping into LIST of Dataframes

import pandas as pd

carbonList=pd.read_html(carbonLink,header=0,flavor='bs4')
forestList=pd.read_html(forestLink,header=0,flavor='bs4')

# getting the Dataframe from list
carbon=carbonList[0]
forest=forestList[0]

# no spaces in column names
carbon.columns=carbon.columns.str.replace(r'\s','',regex=True)
forest.columns=forest.columns.str.replace(r'\s','',regex=True)

#renaming
newCarbonNames={'metrictonnesofCO2':'co2_tonnes','DateofInformation':'Carbon_yearData'}
newForestNames={'%ofGDP':'ForestRevenue_PctGDP', 'DateofInformation':'Forest_yearData'}
carbon.rename(columns=newCarbonNames,inplace=True)
forest.rename(columns=newForestNames,inplace=True)

# dropping
toDrop=['Rank']
carbon.drop(columns=toDrop,inplace=True)
forest.drop(columns=toDrop,inplace=True)

# no spaces in string values (to several columns)
byeSpaces=lambda x: x.str.strip()
carbon.iloc[:,[0,2]]=carbon.iloc[:,[0,2]].apply(byeSpaces)
forest.iloc[:,[0,2]]=forest.iloc[:,[0,2]].apply(byeSpaces)

# keeping year
carbon.Carbon_yearData=carbon.Carbon_yearData.str. extract(pat=r'(\d+)')
forest.Forest_yearData=forest.Forest_yearData.str. extract(pat=r'(\d+)')

Verifying year:

In [29]:
forest[forest.Forest_yearData.str.contains(r'\D')]

,Country,ForestRevenue_PctGDP,Forest_yearData


In [30]:
carbon[carbon.Carbon_yearData.str.contains(r'\D')]

,Country,co2_tonnes,Carbon_yearData


**Before** starting formatting we check the data types:

In [31]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               204 non-null    object 
 1   ForestRevenue_PctGDP  204 non-null    float64
 2   Forest_yearData       204 non-null    object 
dtypes: float64(1), object(2)
memory usage: 4.9+ KB


In [32]:
carbon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          218 non-null    object 
 1   co2_tonnes       218 non-null    float64
 2   Carbon_yearData  218 non-null    object 
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


# String case

This is our string column

In [33]:
carbon.Country

0                                              China
1                                      United States
2                                              India
3                                             Russia
4                                              Japan
                           ...                      
213                                       Antarctica
214    Saint Helena, Ascension, and Tristan da Cunha
215                                             Niue
216                         Northern Mariana Islands
217                                           Tuvalu
Name: Country, Length: 218, dtype: object

In [34]:
# do we have duplicates?
carbon[carbon.duplicated(subset='Country')]

,Country,co2_tonnes,Carbon_yearData


In [35]:
# do we have weird symbols?
carbon[carbon.Country.str.contains(r'[^\w\s]')]

,Country,co2_tonnes,Carbon_yearData
6,"Korea, South",686954000.0,2019
16,Turkey (Turkiye),391792000.0,2019
86,"Korea, North",18465000.0,2019
99,Cote d'Ivoire,11880000.0,2019
137,"Congo, Republic of the",4523000.0,2019
141,"Bahamas, The",3984000.0,2019
150,"Congo, Democratic Republic of the",2653000.0,2019
185,"Gambia, The",606000.0,2019
188,Timor-Leste,538000.0,2019
192,Guinea-Bissau,342000.0,2019


In [36]:
# accents in words:
carbon[carbon.Country.str.contains(r"\w*[\u00C0-\u01DA']\w*")]

,Country,co2_tonnes,Carbon_yearData
99,Cote d'Ivoire,11880000.0,2019


In [37]:
# only ascii
from unidecode import unidecode

carbon['Country']=carbon.Country.apply(unidecode)
forest['Country']=forest.Country.apply(unidecode)

The capitalization is an important step, it may help in later stages when merging data frames:

* str.lower(): all to lowercase.

* str.upper(): ALL TO UPPERCASE.

* str.title(): First Character Of Each Word To Uppercase. 

* str.capitalize(): First character to uppercase and remaining to lowercase.

**You can only apply this if the cells are strings.**

Let's do it:

In [38]:
carbon_test=carbon.copy()
carbon_test['countryname']=carbon_test.Country.str.lower()
carbon_test['COUNTRYNAME']=carbon_test.Country.str.upper()
carbon_test['CountryName']=carbon_test.Country.str.title()
carbon_test

,Country,co2_tonnes,Carbon_yearData,countryname,COUNTRYNAME,CountryName
0,China,1.077325e+10,2019,china,CHINA,China
1,United States,5.144361e+09,2019,united states,UNITED STATES,United States
2,India,2.314738e+09,2019,india,INDIA,India
3,Russia,1.848070e+09,2019,russia,RUSSIA,Russia
4,Japan,1.103234e+09,2019,japan,JAPAN,Japan
...,...,...,...,...,...,...
213,Antarctica,2.800000e+04,2019,antarctica,ANTARCTICA,Antarctica
214,"Saint Helena, Ascension, and Tristan da Cunha",1.300000e+04,2019,"saint helena, ascension, and tristan da cunha","SAINT HELENA, ASCENSION, AND TRISTAN DA CUNHA","Saint Helena, Ascension, And Tristan Da Cunha"
215,Niue,8.000000e+03,2019,niue,NIUE,Niue
216,Northern Mariana Islands,0.000000e+00,2019,northern mariana islands,NORTHERN MARIANA ISLANDS,Northern Mariana Islands


In [39]:
#Let's keep the upper case
carbon['Country']=carbon.Country.str.upper()
forest['Country']=forest.Country.str.upper()

In [40]:
# we can save this:
import os

carbon.to_csv(os.path.join("data","carbon_formatted.csv"),index=False)
forest.to_csv(os.path.join("data","forest_formatted.csv"),index=False)

# Numeric case

If numeric data **looks** good, but the computernhas not recognised it as a number we are still in trouble:

In [41]:
#looks good
carbon.Carbon_yearData

0      2019
1      2019
2      2019
3      2019
4      2019
       ... 
213    2019
214    2019
215    2019
216    2019
217    2019
Name: Carbon_yearData, Length: 218, dtype: object

In [42]:
# not numeric data
carbon.Carbon_yearData.info()

<class 'pandas.core.series.Series'>
RangeIndex: 218 entries, 0 to 217
Series name: Carbon_yearData
Non-Null Count  Dtype 
--------------  ----- 
218 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [43]:
# see ONE element
carbon.Carbon_yearData[0]

'2019'

You can not get any **expected** statistics if the values are not recognised as numeric:

In [44]:
carbon.Carbon_yearData.describe()

count      218
unique       3
top       2019
freq       216
Name: Carbon_yearData, dtype: object

The easiest way is to use the **pd.to_numeric** function in pandas:

In [45]:
# now you get stats for numeric data
pd.to_numeric(carbon.Carbon_yearData).describe()

count     218.000000
mean     2018.958716
std         0.492471
min      2012.000000
25%      2019.000000
50%      2019.000000
75%      2019.000000
max      2019.000000
Name: Carbon_yearData, dtype: float64

Then:

In [46]:
carbon['Carbon_yearData']=pd.to_numeric(carbon.Carbon_yearData)
carbon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          218 non-null    object 
 1   co2_tonnes       218 non-null    float64
 2   Carbon_yearData  218 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 5.2+ KB


In [47]:
forest['Forest_yearData']=pd.to_numeric(forest.Forest_yearData)
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               204 non-null    object 
 1   ForestRevenue_PctGDP  204 non-null    float64
 2   Forest_yearData       204 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.9+ KB


Let's overwrite our previous files

In [48]:
carbon.to_csv(os.path.join("data","carbon_formatted.csv"),index=False)
forest.to_csv(os.path.join("data","forest_formatted.csv"),index=False)

Notice that this works only if the column has already been cleaned:

In [49]:
# a list of values
listValues=[1,2,3,4,'5','x']
dictValues={'someVals':listValues}
# a series
aDataFrame=pd.DataFrame(dictValues)

aDataFrame

,someVals
0,1
1,2
2,3
3,4
4,5
5,x


In [50]:
pd.to_numeric(aDataFrame.someVals)

ValueError: Unable to parse string "x" at position 5

In [51]:
pd.to_numeric(aDataFrame.someVals,errors='coerce')

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    NaN
Name: someVals, dtype: float64

In [52]:
pd.to_numeric(aDataFrame.someVals,errors='ignore')

0    1
1    2
2    3
3    4
4    5
5    x
Name: someVals, dtype: object

In this case, the **coerce** argument would be the best choice.